In [7]:
import os
import tensorflow as tf
import cv2 as cv
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization,Conv2D,MaxPooling2D,Activation,Dropout,Lambda,Dense,Flatten, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.callbacks import EarlyStopping


from keras.utils.vis_utils import plot_model


RuntimeError: The Tensorflow package version needs to be at least 2.11.0 for KerasCV to run. Currently, your TensorFlow version is 2.10.1. Please upgrade with `$ pip install --upgrade tensorflow`. You can use `pip freeze` to check afterwards that everything is ok.

In [3]:
def basic_preProcess(img):
    img = resize_image(img)
    return img

def resize_image(img):
  image_size = (224,224)
  img = cv.resize(img, image_size,interpolation = cv.INTER_LINEAR)
  return img

In [5]:
def color_preProcess(img):
  #insert methods
  # img = to_colorhsv(img)
  # img = to_CIELAB(img)
  # img = to_CIEXYZ(img)
  # img = quantize_image(img,8)

  mean_shift = meanShiftFilter(img)
  grab_cut = grabCut(mean_shift)
  ########################################
  img_rgb = to_RGB(grab_cut)


  return img_rgb

def grabCut (img):
    mask = np.zeros(img.shape[:2], np.uint8)

    rect = (8, 8, 224 -16, 224 -16)
    bgd_model = np.zeros((1, 65), np.float64)
    fgd_model = np.zeros((1, 65), np.float64) 
    cv.grabCut(img, mask, rect, bgd_model, fgd_model, 8, cv.GC_INIT_WITH_RECT)

    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    foreground = img * mask2[:, :, np.newaxis]

    return foreground


def augment_contrast(image, lower=0.4, upper=1.4):
    contrast_factor = np.random.uniform(lower, upper)
    
    adjusted_image = cv.convertScaleAbs(image, alpha=contrast_factor, beta=0)  # alpha controls contrast
    return adjusted_image

def meanShiftFilter(img):

  spatial_radius = 20  # Higher value for more spatial smoothing
  color_radius = 10    # Higher value for more color similarity
  mean_shift_result = cv.pyrMeanShiftFiltering(img, sp=spatial_radius, sr=color_radius)
  return mean_shift_result

def to_colorhsv(img):

  hue_shift = 90
  saturation_scale = 1
  value_scale = 1

  img[..., 1] = np.clip(img[..., 1] * saturation_scale, 0, 255)
  img[..., 2] = np.clip(img[..., 2] * value_scale, 0, 255)
  img[..., 2] = cv.equalizeHist(img[..., 2])

  img = cv.cvtColor(img, cv.COLOR_HSV2BGR)

  return img

def to_CIELAB(img):
  img = cv.cvtColor(img,cv.COLOR_BGR2Lab)
  return img

def to_CIEXYZ(img):
  img = cv.cvtColor(img,cv.COLOR_BGR2XYZ)
  return img

def to_RGB(img):
  img = cv.cvtColor(img,cv.COLOR_BGR2RGB)
  return img

def quantize_image(image, n_colors = 4):
    lab_image = cv.cvtColor(image, cv.COLOR_BGR2Lab)
    pixel_values = lab_image.reshape((-1, 3))
    kmeans = KMeans(n_clusters=n_colors, random_state=42)
    kmeans.fit(pixel_values)
    centers = kmeans.cluster_centers_
    quantized_pixels = centers[kmeans.predict(pixel_values)]
    quantized_image = quantized_pixels.reshape(lab_image.shape).astype(np.uint8)
    quantized_bgr_image = cv.cvtColor(quantized_image, cv.COLOR_Lab2RGB)

    return quantized_bgr_image


In [ ]:
def category_preProcess(img):
  # img = to_grayhsv(img)
  # img = gamma_correct_HSV(img)
  grab_cut = grabCut(img)
  return grab_cut


def to_grayhsv(img):

  saturation_scale = 0
  value_scale = 0.7

  img[..., 1] = np.clip(img[..., 1] * saturation_scale, 0, 255)
  img[..., 2] = np.clip(img[..., 2] * value_scale, 0, 255)
  clahe = cv.createCLAHE(clipLimit=3)
  img[..., 2] = clahe.apply(img[..., 2])


  img = cv.cvtColor(img, cv.COLOR_HSV2BGR)

  # img = cv.GaussianBlur(img,(3,3),0)

  return img

def gamma_correct_HSV(img):

  gamma = 0

  hsv_img = cv.cvtColor(img, cv.COLOR_BGR2HSV)
  H, S, V = cv.split(img)

  lookup_table = np.array([((i / 255.0) ** (1.0 / gamma)) * 255 for i in np.arange(0, 256)]).astype("uint8")

  V_corrected = cv.LUT(V, lookup_table)
  corrected_hsv = cv.merge([H, S, V_corrected])
  corrected_image = cv.cvtColor(corrected_hsv, cv.COLOR_HSV2BGR)

  return corrected_image